In [8]:
# import libraries
# find all sameas links from english to de, fr, es and create three dicts
# for each item in language specific dict
    # find all the triples for language specific subject and uri obj
    # create triple arrays with en subj and replaced obj
    # find all the triples for language specific subject and non uri obj
    # create triple arrays with en subj
    # write triples to file

In [9]:
#import
from SPARQLWrapper import SPARQLWrapper, JSON
import time, os, re

In [10]:
# languages array
lang_arr = ['en','de','es','fr']
# non-english lang array
ne_lang_arr = ['de','es','fr']
# output dir
out_dir = './ada_files/'
# init sparql
sparql = SPARQLWrapper("http://porque.cs.upb.de:3030/dbp-all-2016-10/sparql")

In [11]:
# Obsolete
# create query template
# sameas_qry_tmplt = '''
# PREFIX owl: <http://www.w3.org/2002/07/owl#>
# SELECT 
# ?s1 ?s2 
# WHERE {
# ?s1 owl:sameAs ?s2 . 
# FILTER(STRSTARTS(str(?s1 ), "http://dbpedia.org/resource/" ) &&
#  STRSTARTS(str(?s2 ), "http://%s.dbpedia.org/resource/" ))
# } LIMIT %s
# OFFSET %s
# '''

# uri_trpl_qry = '''
# SELECT 
# ?p ?o
# WHERE { 
# <%s> ?p1 ?o1 .
# FILTER(isURI(?o1))
# }
# '''

# othr_trpl_qry = '''
# SELECT 
# ?p ?o
# WHERE { 
# <%s> ?p1 ?o1 .
# FILTER(!isURI(?o1))
# }
# '''

In [12]:
mapping_file_tmplt = out_dir + "extracted_mappings/extracted_en-%s_links.ttl"

In [13]:

sameas_dict = dict()
for lang in ne_lang_arr:
    cur_map = dict()
    #fetch sameas links
    with open(mapping_file_tmplt%(lang),'r') as mapping_file:
        # read line by line
        while (line_items := mapping_file.readline().rstrip().split()):
            s1 = line_items[0]
            s2 = line_items[2]
            # read results and load mappings to cur_map
            cur_map[s1] = s2
    # load cur_map to sameas_dictraphdb is good. And someone could try out ox
    sameas_dict[lang] = cur_map

In [14]:
print(len(sameas_dict['de']))
print(len(sameas_dict['es']))
print(len(sameas_dict['fr']))

1139521
1005148
1359645


In [59]:
LITERAL_TYPE = 'L'
URI_TYPE = 'U'
OTHER_TYPE = 'O'

In [60]:
def identify_and_write(s, p, o, link_map, file_arr):
    out_file = None
    out_line = None
    # find if subj linked
    subj_lnkd = s in link_map
    # type of obj is other
    o_type = OTHER_TYPE
    o_lnkd = False
    
    if re.match('^<.+>$', o):
        o_type = URI_TYPE
        # find if obj linked
        o_lnkd = o in link_map
    elif re.match('^".+".*', o):
        o_type = LITERAL_TYPE
    #print('Object:', o, '\tObj Type', o_type, '\tObj linked', o_lnkd)
    # if both linked, write to 0
    if(subj_lnkd and o_lnkd):
        out_file = file_arr[0]
        out_line = link_map[s] + ' ' + p + ' ' + link_map[o] + ' .\n'
    elif((not subj_lnkd) and o_lnkd):
        out_file = file_arr[2]
        out_line = s + ' ' + p + ' ' + link_map[o] + ' .\n'
    # if subj linked
    elif(subj_lnkd and (not o_lnkd)):
        # if obj is literal write to 1
        if(o_type == LITERAL_TYPE):
            out_file = file_arr[1]
        # rest, write to 2
        else:
            out_file = file_arr[2]
        out_line = link_map[s] + ' ' + p + ' ' + o + ' .\n'
    # if subj and obj both not linked
    else:
        # if obj is literal, write to 4
        if(o_type == LITERAL_TYPE):
            out_file = file_arr[4]
        # is obj is not literal, write to 3
        else:
            out_file = file_arr[3]
            #print('This object should not be literal:', o, ' type:', o_type)
            
        out_line = s + ' ' + p + ' ' + o + ' .\n'
    out_file.write(out_line)

In [61]:
enrich_dir ='/data-disk/kg-fusion/enrichment-files/'
triples_dir = "/data-disk/kg-fusion/"
# For each language
for lang in ne_lang_arr:
    link_map = sameas_dict[lang]
    directory = os.fsencode(triples_dir + lang + '/')
    out_dir = enrich_dir + lang + '/'
    fully_lnkd_fp = out_dir + 'fully_lnkd.nt'
    lnkd_ltrls_fp = out_dir + 'lnkd_ltrls.nt'
    part_lnkd_fp = out_dir + 'part_lnkd.nt'
    non_lnkd_fp = out_dir + 'non_lnkd.nt'
    nl_ltrls_fp = out_dir + 'nl_ltrls.nt'
    
    # create directories if not exists
    os.makedirs(os.path.dirname(fully_lnkd_fp), exist_ok=True)
    
    with open(fully_lnkd_fp,'w') as fully_lnkd, \
        open(lnkd_ltrls_fp,'w') as lnkd_ltrls, \
        open(part_lnkd_fp,'w') as part_lnkd, \
        open(non_lnkd_fp,'w') as non_lnkd, \
        open(nl_ltrls_fp,'w') as nl_ltrls:
        file_arr = [fully_lnkd, lnkd_ltrls, part_lnkd, non_lnkd, nl_ltrls]
        # For each file
        for file in os.listdir(directory):
            filename = os.fsdecode(file)
            if filename.endswith(".ttl"):
                print('Processing',filename)
                with open(directory+file,'r') as cur_file:
                    # For each line
                    while (line := cur_file.readline().rstrip()):
                        if(re.match('^#.*',line[0])):
                            continue
                        # extract items
                        line_items = re.match('(<.+?>)\s(<.+?>)\s(.+)\s\\.', line).groups()
                        # Find mappings
                        s = line_items[0]
                        p = line_items[1]
                        o = line_items[2]
                        #print(s,p,o)
                        identify_and_write(s, p, o, link_map, file_arr)
                            
            else:
                continue

Processing mappingbased_literals_de.ttl
Processing commons_page_links_de.ttl
Processing category_labels_de.ttl
Processing instance_types_de.ttl
Processing skos_categories_de.ttl
Processing persondata_de.ttl
Processing interlanguage_links_de.ttl
Processing long_abstracts_de.ttl
Processing labels_de.ttl
Processing article_categories_de.ttl
Processing short_abstracts_de.ttl
Processing mappingbased_objects_de.ttl
Processing disambiguations_de.ttl
Processing short_abstracts_es.ttl
Processing instance_types_es.ttl
Processing commons_page_links_es.ttl
Processing interlanguage_links_es.ttl
Processing persondata_es.ttl
Processing skos_categories_es.ttl
Processing mappingbased_objects_es.ttl
Processing long_abstracts_es.ttl
Processing mappingbased_literals_es.ttl
Processing category_labels_es.ttl
Processing article_categories_es.ttl
Processing disambiguations_es.ttl
Processing labels_es.ttl
Processing instance_types_fr.ttl
Processing article_categories_fr.ttl
Processing french_population_fr.ttl


In [62]:
print('Files parting completed.')

Files parting completed.
